In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session/

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

sns.set(rc={'figure.figsize': (12,10)})
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_Score = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
df_Score

In [ ]:
df_train.head()

# EDA

In [ ]:
df_train.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
print('Female Survival : ', df_train['Survived'][df_train['Sex'] == 'female'].value_counts(normalize = True)[1]*100)
print('Male Survival   : ', df_train['Survived'][df_train['Sex'] == 'male'].value_counts(normalize = True)[1]*100)
sns.barplot(x = 'Sex', y = 'Survived', data = df_train)

In [ ]:
df_train['Age'].fillna(-0.5, inplace = True)
df_test['Age'].fillna(-0.5, inplace = True)
bins = [-1, 0, 5, 12, 19, 24, 35, 60, np.inf]
labels = ['Unkown', 'Baby', 'Child', 'Teenager', 'Young', 'Young Adult', 'Adult', 'Senior']

df_train['AgeGroup'] = pd.cut(df_train['Age'], bins, labels = labels)
df_test['AgeGroup'] = pd.cut(df_test['Age'], bins, labels = labels)
sns.barplot(x = 'AgeGroup', y = 'Survived', data = df_train)

In [ ]:
train = df_train[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 
          #'Ticket', 'Fare', 'Cabin', 
          'Embarked', 'AgeGroup']]
test = df_test[['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 
          #'Ticket', 'Fare', 'Cabin', 
          'Embarked', 'AgeGroup']]

In [ ]:
for c in train['Pclass'].unique():
    print(c, ' Class Survival : ', df_train['Survived'][df_train['Pclass'] == c].value_counts(normalize = True)[1]*100)
sns.barplot(x = 'Pclass', y = 'Survived', data = df_train)

In [ ]:
print('No Siblings : ', train[train['SibSp'] == 0 ].shape[0])
print('One Siblings : ', train[train['SibSp'] == 1].shape[0])
print('Two Siblings : ', train[train['SibSp'] == 2].shape[0])
print('Two Siblings : ', train[train['SibSp'] == 3].shape[0])
print('Two Siblings : ', train[train['SibSp'] == 4].shape[0])
print('Two Siblings : ', train[train['SibSp'] == 5].shape[0])
print('Two Siblings : ', train[train['SibSp'] == 8].shape[0])
sns.barplot(x = 'SibSp', y = 'Survived', data = train)


In [ ]:
for i in train['SibSp'].unique():

    print('Survival percentage of', i ,'Siblings :', train['Survived'][train['SibSp'] == i].value_counts(normalize = True)*100)

In [ ]:
df_test.columns

In [ ]:
test

In [ ]:
train

# Dealing with Missing Values

In [ ]:
print('Null Values in Train data :\n ', train.isnull().sum())
print('Null Values in Test data :\n ', test.isnull().sum())

In [ ]:
for e in train['Embarked'].unique():
    print('Number of Passenger Embarked on', e ,  train[train['Embarked'] == e].shape[0])


In [ ]:
train['Embarked'].fillna('S', inplace = True)
print(train['Embarked'].isnull().sum())

In [ ]:
print('Null Values in Train data :\n ', train.isnull().sum())
print('Null Values in Test data :\n ', test.isnull().sum())

In [ ]:
combine = [train, test]
for data in combine:
    data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
pd.crosstab(train['Title'], train['Sex'])

In [ ]:
test['Title'].unique()

In [ ]:
for data in combine:
    data['Title'] = data['Title'].replace(['Capt', 'Col', 'Don','Dona', 'Dr', 'Jonkheer', 'Major','Rev'], 'Rare')
    data['Title'] = data['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    data['Title'] = data['Title'].replace( 'Mme', 'Mrs')
    data['Title'] = data['Title'].replace(['Mlle', 'Ms'], 'Miss')

In [ ]:
train[['Title', 'Survived' ]].groupby('Title').mean()

In [ ]:
title_mapping = {'Miss' : 1, 'Mr': 2, 'Mrs' : 3, 'Master' : 4, 'Royal': 5, 'Rare': 6}
for data in combine:
    data['Title'] = data['Title'].map(title_mapping)
train.head()


In [ ]:
train['AgeGroup'].unique()

In [ ]:
miss_age = train[train['Title'] == 1]['AgeGroup'].mode() #Teenager
mr_age = train[train['Title'] == 2]['AgeGroup'].mode() #Young Adult
mrs_age = train[train['Title'] == 3]['AgeGroup'].mode() #Adult
master_age = train[train['Title'] == 4]['AgeGroup'].mode() #Baby
royal_age = train[train['Title'] == 5]['AgeGroup'].mode() #Adult
rare = train[train['Title'] == 6]['AgeGroup'].mode() #Adult

age_title_map = {1: 'Young Adult', 2 : 'Young', 3 : 'Adult', 4 : 'Baby', 5 : 'Child', 6 : 'Teenager', 7: 'Senior'}

for i in range(len(train['AgeGroup'])):
    if train['AgeGroup'][i] == 'Unkown':
        train['AgeGroup'][i] = age_title_map[train['Title'][i]]
        
for i in range(len(test['AgeGroup'])):
    if test['AgeGroup'][i] == 'Unkown':
        test['AgeGroup'][i] = age_title_map[test['Title'][i]]

In [ ]:
train['AgeGroup'].unique()

In [ ]:
miss_age, mr_age, mrs_age, master_age, royal_age, rare

# Modelling

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
transform = ['Sex', 'AgeGroup', 'Embarked']

for i in transform:
    train[i] = le.fit_transform(train[i])

In [ ]:
for j in transform:
    test[j] = le.fit_transform(test[j])

In [ ]:
test

In [ ]:
x_train = train.drop(['PassengerId', 'Survived', 'Name'], axis = 1)
y_train = train['Survived']
x_test = test.drop(['PassengerId', 'Name'], axis = 1)
y_test = df_Score['Survived']

In [ ]:
x_test

# Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

# Classificaiton

In [ ]:
logreg_classifier = LogisticRegression()

# Training

In [ ]:
logreg_classifier.fit(x_train, y_train)

# Predictions

In [ ]:
logreg_classifier_predictions = logreg_classifier.predict(x_test)

In [ ]:
logreg_classifier_predictions

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

logreg_acc = accuracy_score(y_test, logreg_classifier_predictions)

In [ ]:
print('Logistic Regression : ', round(logreg_acc*100,2))

In [ ]:
ids = test['PassengerId']
output = pd.DataFrame({'PassengerId' : ids, 'Survived' : logreg_classifier_predictions})
output.to_csv('Submission.csv', index = False)